## Retrieving weather data with API

In [ ]:
from datetime import datetime
import pytz

In [ ]:
# here apply your API key
API_key = os.environ.get('API_key_weather') 

In [ ]:
def get_weather_data (cities):
    
    tz = pytz.timezone('Europe/Berlin')
    now = datetime.now().astimezone(tz)
    
    
    weather_dict = {'city': [],
                'country': [],
                'forecast_time': [],
                'outlook': [],
                'detailed_outlook': [],
                'temperature': [],
                'temperature_feels_like': [],
                'clouds': [],
                'rain': [],
                'snow': [],
                'wind_speed': [],
                'wind_deg': [],
                'humidity': [],
                'pressure': [],
                'information_retrieved_at': []}
    
    
    for city in cities:
        url = (f"http://api.openweathermap.org/data/2.5/forecast?q={city}&appid={API_key}&units=metric")
        response = requests.get(url)
        json = response.json()

        for i in json['list']:
            weather_dict['city'].append(json['city']['name'])
            weather_dict['country'].append(json['city']['country'])
            weather_dict['forecast_time'].append(i['dt_txt'])
            weather_dict['outlook'].append(i['weather'][0]['main'])
            weather_dict['detailed_outlook'].append(i['weather'][0]['description'])
            weather_dict['temperature'].append(i['main']['temp'])
            weather_dict['temperature_feels_like'].append(i['main']['feels_like'])
            weather_dict['clouds'].append(i['clouds']['all'])
        
            try:
                weather_dict['rain'].append(i['rain']['3h'])
            except:
                weather_dict['rain'].append('0')
            try:
                weather_dict['snow'].append(i['snow']['3h'])
            except:
                weather_dict['snow'].append('0')
            
                weather_dict['wind_speed'].append(i['wind']['speed'])
                weather_dict['wind_deg'].append(i['wind']['deg'])
                weather_dict['humidity'].append(i['main']['humidity'])
                weather_dict['pressure'].append(i['main']['pressure'])
                weather_dict['information_retrieved_at'].append(now.strftime("%d/%m/%Y %H:%M:%S"))
            
            
            
    return pd.DataFrame(weather_dict)

In [ ]:
# here, the same list from the web_scraping cities info, can be added
get_weather_data(['Berlin'])

## Using json_nomalize

In [ ]:
def get_weather_norm(cities):
    
    
    df_list = []

    for city in cities:
        url = (f"http://api.openweathermap.org/data/2.5/forecast?q={city}&appid={API_key}&units=metric")
        response = requests.get(url)
        json = response.json()

        json_norm_df = pd.json_normalize(json['list'],
                                    record_path=['weather'],
                                    meta=['dt_txt', ['main', 'temp'], ['main', 'feels_like'], ['clouds', 'all'], ['rain', '3h'], ['snow', '3h'], ['wind', 'speed'], ['wind', 'deg'], ['main', 'humidity'], ['main', 'pressure']],
                                    errors='ignore')
        json_norm_df.drop(columns=['id', 'icon'], inplace=True)
        json_norm_df.rename(columns={'main': 'outlook',
                                 'description': 'detailed_outlook',
                                 'dt_txt': 'forecast_time',
                                 'main.temp': 'temperature',
                                 'main.feels_like': 'temperature_feels_like',
                                 'clouds.all': 'clouds',
                                 'rain.3h': 'rain',
                                 'snow.3h': 'snow',
                                 'wind.speed': 'wind_speed',
                                 'wind.deg': 'wind_deg',
                                 'main.humidity': 'humidity',
                                 'main.pressure': 'pressure',},
                        inplace=True)
        json_norm_df.insert(0, 'city', json['city']['name'])
        json_norm_df.insert(1, 'country', json['city']['country'])
        json_norm_df['information_retrieved_at'] = now.strftime("%d/%m/%Y %H:%M:%S")
        json_norm_df[['city', 'country', 'forecast_time', 'outlook', 'detailed_outlook',
              'temperature', 'temperature_feels_like', 'clouds', 'rain', 'snow',
              'wind_speed', 'wind_deg', 'humidity', 'pressure',
              'information_retrieved_at']]
        df_list.append(json_norm_df)
        
        
    return pd.concat(df_list, ignore_index=True)

In [ ]:
# here, the same list from the web_scraping cities info, can be added
get_weather_data(['Berlin'])